# Образец результата

прогноз на апрель - август 2023, сделанный на данных до марта включительно

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os
os.chdir(os.path.dirname(os.getcwd()))

from datetime import datetime
import pandas as pd

In [2]:
data_root = os.path.join('data', 'main')

In [3]:
# Факт на задание
fact = pd.read_csv(os.path.join(data_root, "fact_train_test.csv"), sep=";", decimal=",", encoding="windows-1251")
fact["period"] = fact["period"].astype("datetime64[ns]")
# fact - уже обрезанная выборка

# Пример прогноза - наивный, копия последнего факта
result = []
last_known_fact_month = fact["period"].max()
test_periods = pd.date_range(start=last_known_fact_month, periods=6, freq='MS', inclusive='right')
for period in test_periods:
    print(period)
    month_forecast = fact[fact["period"] == last_known_fact_month].reset_index(drop=True)
    month_forecast["period"] = period
    result.append(month_forecast)
result = pd.concat(result).reset_index(drop=True)
result.rename(columns={"real_wagon_count": "forecast_wagon_count", "real_weight": "forecast_weight"}, inplace=True)
result.to_csv(os.path.join(data_root, "forecast_example.csv"), index=False, sep=";", decimal=",", encoding="windows-1251")


2023-04-01 00:00:00
2023-05-01 00:00:00
2023-06-01 00:00:00
2023-07-01 00:00:00
2023-08-01 00:00:00


# Оценка результата

In [4]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
# import os
# os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd
import numpy as np


In [8]:
def add_master_data_mappings(df: pd.DataFrame) -> pd.DataFrame:
    # = Пути к справочникам - откорректировать если в реальной системе будут лежать по другому адресу =
    client_mapping_file = os.path.join(data_root, "client_mapping.csv")
    freight_mapping_file = os.path.join(data_root, "freight_mapping.csv")
    station_mapping_file = os.path.join(data_root, "station_mapping.csv")

    # Клиент - холдинг
    client_mapping = pd.read_csv(
        client_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(df, client_mapping, how="left", on="client_sap_id")

    # Груз
    freight_mapping = pd.read_csv(
        freight_mapping_file, sep=";", decimal=",", encoding="windows-1251"
    )
    df = pd.merge(df, freight_mapping, how="left", on="freight_id")

    # Станции
    station_mapping = pd.read_csv(
        station_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("sender_"),
        how="left",
        on="sender_station_id",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("recipient_"),
        how="left",
        on="recipient_station_id",
    )

    return df


def evaluate(fact: pd.DataFrame, forecast: pd.DataFrame, public: bool = True) -> float:
    # = Параметры для расчета метрики =
    accuracy_granularity = [
        "period",
        "rps",
        "holding_name",
        "sender_department_name",
        "recipient_department_name",
    ]
    fact_value, forecast_value = "real_wagon_count", "forecast_wagon_count"
    if public:
        metric_weight = np.array([0.0, 1.0, 0.0, 0.0, 0.0])
    else:
        metric_weight = np.array([0.1, 0.6, 0.1, 0.1, 0.1])

    # = Собственно расчет метрик =
    # 1. Добавляем сущности верхних уровней гранулярности по справочникам
    fact = add_master_data_mappings(fact)
    forecast = add_master_data_mappings(forecast)

    # 2. Расчет KPI
    compare_data = pd.merge(
        fact.groupby(accuracy_granularity, as_index=False)[fact_value].sum(),
        forecast.groupby(accuracy_granularity, as_index=False)[forecast_value].sum(),
        how="outer",
        on=accuracy_granularity,
    ).fillna(0)
    # Против самых хитрых - нецелочисленный прогноз вагоноотправок не принимаем
    compare_data[fact_value] = np.around(compare_data[fact_value]).astype(int)
    compare_data[forecast_value] = np.around(compare_data[forecast_value]).astype(int)

    # 3. Рассчитаем метрики для каждого месяца в выборке
    compare_data["ABS_ERR"] = abs(
        compare_data[forecast_value] - compare_data[fact_value]
    )
    compare_data["MAX"] = abs(compare_data[[forecast_value, fact_value]].max(axis=1))
    summary = compare_data.groupby("period")[
        [forecast_value, fact_value, "ABS_ERR", "MAX"]
    ].sum()
    summary["Forecast Accuracy"] = 1 - summary["ABS_ERR"] / summary["MAX"]

    # 4. Взвесим метрики отдельных месяцев для получения одной цифры score
    score = (
        summary["Forecast Accuracy"].sort_index(ascending=True) * metric_weight
    ).sum()

    return score


def calc_score_public(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=True)


def calc_score_private(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=False)



In [11]:
forecast.head()

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,forecast_weight,forecast_wagon_count
0,2023-04-01,0,0,0,2275,3291,23976,26423,27983,73.0,1
1,2023-04-01,0,0,0,2275,3294,23976,26423,27983,146.0,2
2,2023-04-01,0,0,0,2275,3294,23976,24516,27983,876.0,12
3,2023-04-01,0,0,0,-1,349,38725,38966,27437,473.0,7
4,2023-04-01,0,0,0,-1,349,38754,38966,26664,63.0,1


In [12]:
forecast = add_master_data_mappings(forecast)

In [14]:
forecast.head()

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,forecast_weight,forecast_wagon_count,holding_name,freight_group_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name
0,2023-04-01,0,0,0,2275,3291,23976,26423,27983,73.0,1,13.0,20,77,31,124,5
1,2023-04-01,0,0,0,2275,3294,23976,26423,27983,146.0,2,13.0,20,77,31,124,5
2,2023-04-01,0,0,0,2275,3294,23976,24516,27983,876.0,12,13.0,20,77,31,147,31
3,2023-04-01,0,0,0,-1,349,38725,38966,27437,473.0,7,NaN,10,23,6,23,6
4,2023-04-01,0,0,0,-1,349,38754,38966,26664,63.0,1,NaN,10,23,6,23,6


In [17]:
data_info = pd.read_excel(os.path.join(data_root, 'Описание данных для хакатона - прогноз ЖД перевозок.xlsx'))

In [18]:
data_info

,Часть истории перевозок компаниии ПГК с января 2017 г. до марта 2023 г. включительно,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,Поле,Тип данных,Описание
2,period,date,Месяц предъявления отправки к перевозке в форм...
3,rps,integer,Род Подвижного Состава
4,podrod,integer,Подрод Подвижного Состава
5,filial,integer,"Филиал ""ПГК"""
6,client_sap_id,integer,Код отправителя
7,freight_id,integer,Код груза
8,sender_station_id,integer,Код пункта отправления
9,recipient_station_id,integer,Код пункта назначения


In [9]:
# = Примеры файлов для проверки =
validation_file = os.path.join(data_root, "fact_validation.csv")
forecast_file = os.path.join(data_root, "forecast_example.csv")

# Валидационный датасет
# fact = pd.read_csv(validation_file, sep=";", decimal=",", encoding="windows-1251")
# print("Валидационный датасет:", fact.shape)
# Прогноз
forecast = pd.read_csv(forecast_file, sep=";", decimal=",", encoding="windows-1251")
# print("Прогноз:", forecast.shape)

# Скорим
score_public = calc_score_public(fact, forecast)
score_private = calc_score_private(fact, forecast)
print(f"Public score: {score_public}")
print(f"Private score: {score_private}")


ValueError: You are trying to merge on datetime64[ns] and object columns for key 'period'. If you wish to proceed you should use pd.concat